# threadingでコントロールしながら距離を取得

In [1]:
# -*- coding: utf-8 -*-
from ipywidgets import *
from IPython.display import display
import threading
import time
import smbus
import sys
import autodrive
reload(autodrive)
import LVEZ1
reload(LVEZ1)
import csv
import RPi.GPIO as GPIO
import pandas as pd

LV = LVEZ1.Dist()
bus = smbus.SMBus(1)
D = autodrive.Car()

def ard_map(self, x, in_min, in_max, out_min, out_max):
        return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min


Bad key "+ backend" on line 39 in
/etc/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://matplotlib.sf.net/_static/matplotlibrc or from the matplotlib source
distribution

Bad key "- backend" on line 38 in
/etc/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://matplotlib.sf.net/_static/matplotlibrc or from the matplotlib source
distribution


In [2]:
# dlist1 = []
# f = open('distdata.csv', 'w')
# csvWriter = csv.writer(f)

dlist_front = []
dlist_frontL = []
dlist_frontR = []
dlist_behindR = []
dlist_back = []
Lmotor=[]
Rmotor=[]

class getdist(threading.Thread):
 
    def __init__(self):
         threading.Thread.__init__(self)

    def run(self):
        while True:
            f,fr,l,br,ba = LV.Measure(20),LV.Measure(21),LV.Measure(19),LV.Measure(26),LV.Measure(6)
#             print 'front : %d , frontR : %d , Left : %d , BackR : %d , Back: %d' % (f,fr,l,br,ba)
            sys.stdout.write('\r front : %d , frontR : %d , Left : %d , BackR : %d , Back: %d' % (f,fr,l,br,ba))
            sys.stdout.flush()
            if f  < 30:
                D.brake()
            dlist_front.append(f)
            time.sleep(0.01)
            dlist_frontR.append(fr)
            time.sleep(0.01)
            dlist_frontL.append(l)
            time.sleep(0.01)
            dlist_behindR.append(br)
            time.sleep(0.01)
            dlist_back.append(ba)
            
            time.sleep(0.3)
            
 
th = getdist()
th.setDaemon(True)
th.start()

try:
    button1 = widgets.Button(description="Go ahead!")
    button2 = widgets.Button(description="Back!")
    button3 = widgets.Button(description="Stop!!")
    button4 = widgets.Button(description="Left Trun")
    button5 = widgets.Button(description="Right Trun")

    display(button1)
    display(button2)
    display(button3)
    display(button4)
    display(button5)

    def go_ahead(b):
        if handle.value == 0:
            D.forward(speed.value)
#             Lmotor.append(speed.value)
#             Rmotor.append(speed.value)

        elif handle.value > 0:
            D.right(speed.value,handle.value)
#             Lmotor.append(speed.value)
#             Rmotor.append(speed.value)

        elif handle.value < 0:
            D.left(speed.value,-1 * handle.value)

    def back(b):
        D.back(speed.value)

    def stop(b):
        D.brake()

    def l_turn(b):
        D.R_forward(speed.value)
        D.L_back(speed.value)

    def r_turn(b):
        D.L_forward(speed.value)
        D.R_back(speed.value)

    speed = IntSlider(description='Speed')
    display(speed)
    handle = IntSlider(min=-10, max=10, value=0,description='Handle_pitch')
    display(handle)
    button1.on_click(go_ahead)
    button2.on_click(back)
    button3.on_click(stop)
    button4.on_click(l_turn)
    button5.on_click(r_turn)

except KeyboardInterrupt:
#     print(dlist_front)
    th.stop()
#     csvWriter.writerow(dlist_front) 
#     writer = csv.writer(f, lineterminator='\n')
    GPIO.cleanup()
#     f.close()

LVEZ1.py:16: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(GPIO_TRIGECHO,GPIO.OUT)


 front : 39 , frontR : 26 , Left : 60 , BackR : 23 , Back: 49

In [3]:
direction_list = [dlist_front, dlist_frontL, dlist_frontR, dlist_behindR, dlist_back]

 front : 42 , frontR : 26 , Left : 51 , BackR : 20 , Back: 49

In [4]:
df_list = [pd.DataFrame(frame) for frame in direction_list]

 front : 42 , frontR : 26 , Left : 51 , BackR : 20 , Back: 49

In [5]:
type(df_list[0])

pandas.core.frame.DataFrame

 front : 39 , frontR : 26 , Left : 51 , BackR : 23 , Back: 49

In [6]:
df = df_list[0]
del df_list[0]
for data in df_list:
    df = pd.concat([df, data],axis = 1)
    
df.columns = ['dlist_front', 'dlist_frontL', 'dlist_frontR', 'dlist_behindR', 'dlist_back']

 front : 42 , frontR : 26 , Left : 51 , BackR : 20 , Back: 50

In [10]:
df.to_csv("dataset/distdata.csv")

 front : 49 , frontR : 26 , Left : 51 , BackR : 20 , Back: 16